# Simulate synapses onto aCC

In [1]:
%matplotlib notebook

import csv
from neuron import h, gui
from neuron.units import ms, mV
from matplotlib import pyplot

NEURON: unable to open font "*helvetica-medium-r-normal*--14*", using "fixed"


## Import Bokeh for graphs

In [28]:
from bokeh.io import output_notebook
import bokeh.plotting as plt
from bokeh.models.ranges import DataRange1d
import numpy as np
from bokeh.io import export_svg

output_notebook()

Loading BokehJS ...

## Load morphology and other components

In [2]:
# load channels
h('''{load_file("../chan-DmNaT-ODowd.hoc")
      load_file("../chan-DmNaP-DmNav10.hoc")
      load_file("../chan-DmKdr-Marley.hoc")
      load_file("../chan-DmKA-Marley.hoc")
     }''')

# load morphology
h('{load_file("../neuron-morph-ext-axon-2pieces-chans-ext-axon-70um.hoc")}')

# enable or disable voltage clamp
h('load_file("../vclamp_soma_-60mV.ses")')

	0 
	0 
	0 
	0 
	0 
	1 


1

## Initialization

In [3]:
# load common funcs
h('load_file("../fitfuncs.hoc")')

#h('print_elec_cell()') #fix this 

# calc morph stats
h('load_file("../stats.hoc")')

file_name = "ext-axon-70um-sim-synapses"

# save state in this file
h.state_file_name = "state-neuron-act+elec+ext-axon-2piece-chans-" + file_name + ".bin"

# load state first because it overrides parameters
state_w_syn = "state-neuron-act+elec+ext-axon-2piece-chans-" + file_name + "-synapse-saved.bin"
h.restoreStateFromFile(state_w_syn)

# small adjustments

# increase VC duration
h.VClamp[0].dur[0] = 1000

	1 
Secname	L (um)	Diam (um)	Area [um^2]	Ri [MO]		Cm [pF]	g_pas [nS]
axon:	37.61	1.14	134.11	35.12	1.34	0.00
soma:	6.98	5.50	120.56	0.28	1.21	0.00
bottom dend:	996.91	0.78	2430.62	1991.49	24.31	0.02
botdend - axon:	959.10	0.76	2292.50	1993.51	22.92	0.02
topdend:	366.00	0.73	837.85	829.37	8.38	0.01
ext. axon:	570.00	0.71	1275.49	1351.81	12.75	1.53
	1 


In [4]:
# Dictionary for netcon objects indexed by section number that hold the synapse and netstim objects
syns = {}

class Synapse():
    
    def __init__(self, name, sec, syn, ns, nc):
        self.name = name
        self.sec = sec
        self.syn = syn
        self.ns = ns
        self.nc = nc
        
    def set_stim(self, ps, weight = .00018, tau = 5, interval = 10, magepsc = 20, syne = 0, number = 1, start = 10):
        """Set parameters of specified synapse"""
        nc = self.nc
        syn = self.syn
        ns = self.ns
        print(f"Setting: weight={weight}, tau={tau}, interval={interval}, magepsc={magepsc}, syne={syne}, number={number}, start={start}")
        syn.tau = tau
        syn.e = syne
        ns.interval = interval
        ns.number = number
        ns.start = start
        ns.noise = 0
        nc.weight[0] = weight * magepsc
        ps.color(2, self.sec(0.5))

def create_syns(*secs):
    """Factory method to create synapse, NetStim, and NetCon objects"""
    for sec in secs:
        print(f"Creating synapse on {sec}")
        syn = h.ExpSyn(sec(0.5))
        ns = h.NetStim(sec(0.5))
        nc = h.NetCon(ns, syn)
        nc.delay = 0
        syn_obj = Synapse(str(sec), sec, syn, ns, nc)
        syns[str(sec)] = syn_obj

'''
 * synapses:
 * top dend: 685, 524, 520, 626
 * bot dend: 205, 357, 464, 588, 513, 48
'''

create_syns(h.dendrite[685], h.dendrite[524], h.dendrite[520], h.dendrite[626], h.dendrite[205], h.dendrite[357], 
            h.dendrite[464], h.dendrite[588], h.dendrite[513], h.dendrite[48])
#set_syn_pars()                  # use defaults

Creating synapse on dendrite[685]
Creating synapse on dendrite[524]
Creating synapse on dendrite[520]
Creating synapse on dendrite[626]
Creating synapse on dendrite[205]
Creating synapse on dendrite[357]
Creating synapse on dendrite[464]
Creating synapse on dendrite[588]
Creating synapse on dendrite[513]
Creating synapse on dendrite[48]


# Experiments

## create shape plot

In [5]:
ps = h.PlotShape(0) # don't show a default view
ps.show(0)         # show diameters
ps.view(30,10,55,87, 300, 100, 500, 700)
h.load_file('stdrun.hoc')

1.0

In [21]:
vc_current = h.Vector().record(h.VClamp[0]._ref_i)  # Voltage clamp current vector
v_soma = h.Vector().record(h.soma[0](0.5)._ref_v)  # Membrane potential vector
t = h.Vector().record(h._ref_t)  # Time stamp vector

## Run a short simulation and save after steady state is reached

In [ ]:
# warning! if state changes (new synapses, etc) restoring saved state file will cause a crash!
h.finitialize(-60 * mV)
h.continuerun(100 * ms)
# rewind and save
h.finitialize()
h.saveState()
print("State saved to: " + h.state_file_name)

## stimulate one synapse with varying magnitudes

In [22]:
syn = syns['dendrite[685]']

In [23]:
h.restoreState()
syn.set_stim(ps = ps, weight = .0018, tau=5, interval = 0, number = 1)
h.finitialize(-60 * mV)
h.continuerun(60 * ms)

Reading state from file.
Setting: weight=0.0018, tau=5, interval=0, magepsc=20, syne=0, number=1, start=10


0.0

In [33]:
f = plt.figure(x_axis_label="t (ms)", y_axis_label="v (mV)")
f.line(np.array(t), np.array(vc_current), line_width=2)
f.x_range = DataRange1d(start = 9, end = 50)
f.y_range = DataRange1d(start = -0.25, end = 0)
plt.show(f)

In [34]:
# save as SVG
f.output_backend = "svg"
export_svg(f, filename="single-input-dendrite-685.svg")

['single-input-dendrite-685.svg']

In [ ]:
syn2 = syns['dendrite[48]']

In [ ]:
#syn.set_stim(ps = ps, weight = 0, tau=5, interval = 0, number = 1)
h.restoreState()
#syn.set_stim(ps = ps, weight = .0018, tau=5, interval = 0, number = 1)
syn2.set_stim(ps = ps, weight = .0018, tau=5, interval = 0, number = 1)
h.finitialize()
h.continuerun(60 * ms)

In [ ]:
h.steady_state.save()
h('state_fobj = new File("state-neuron-act+elec+ext-axon-2piece-chans-ext-axon-70um-sim-synapses.bin")')
h.steady_state.fwrite(h.state_fobj)

In [ ]:
syns

In [10]:
f = plt.figure(x_axis_label="t (ms)", y_axis_label="v (mV)")
f.line(np.array(t), np.array(v), line_width=2)
plt.show(f)

### Save as SVG (requires installing selenium)

['plot.svg']

## To Do
- Switch to jupyter graphs since I can't reset the Interviews graphs with continuerun()

In [25]:
f.x_range

DataRange1d(id='p1913', ...)